## Make the connection to the appropriate data in Google Big Query

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import random
import sqlite3

In [2]:
service_path = "C:\\Users\\Craig\\Documents\\000000000 UofM fall 2021\\BMIS670 Chandler AppliedDataAnalytics\\Wedge project\\"
# THE PATH TO THE DATA READY FOR ANALYSIS
service_file = 'wedge-project-fall2021-92691-f3182a53adb6.json' # this is my authentication information  
gbq_proj_id = 'wedge-project-fall2021-92691'  # this is my project_id
gbq_dataset_id = 'Wedge_FULL'

In [3]:
# Get my credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# Creating a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Query 1

1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items

In [4]:
task3_nbr1 = '''SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(HOUR FROM datetime)) AS Hour,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-fall2021-92691.Wedge_FULL.transArchive*`
    
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour'''
date_hour_transactions = client.query(task3_nbr1,location="US")

### set up headers

In [5]:
headers = ["Date","Hour","Sales","Transactions","Items"]

### Create text file of query results

In [6]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

with open("daily_hourly_transactions.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in date_hour_transactions:
         outfile.write(",".join([str(item) for item in row])+ '\n')
            
end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process            

10.758096600000002


### Write to a table in SQLite

In [7]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

db = sqlite3.connect('Wedge_Task3.db') # connect to the new database
c = db.cursor()

# input the results of the first query into the WedgeTask3 database as the table Daily_Hourly_Sales
input_file1 = "daily_hourly_transactions.txt"

c.execute('''DROP TABLE IF EXISTS Daily_Hourly_Sales''') # remove the table if it already exists
# create the table in the connected database and set up the schema
c.execute('''CREATE TABLE Daily_Hourly_Sales (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')
# input the Sales by Date by Hour text data into the established table in the database
with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO Daily_Hourly_Sales (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion
db.commit()

end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process

0.17895629999999585


## Query 2

2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 

In [8]:
task3_nbr2 = '''SELECT card_no,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-fall2021-92691.Wedge_FULL.transArchive*`

    WHERE department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY card_no, Year, Month
    ORDER BY card_no, Year, Month'''

owner_year_month = client.query(task3_nbr2,location="US")

### set up headers

In [9]:
headers = ["Owner_ID","Year","Month","Sales","Transactions","Items"]

### Create text file of query results

In [10]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

with open("Owner_year_month_trans.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in owner_year_month:
         outfile.write(",".join([str(item) for item in row])+ '\n')
            
end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process            

58.27829319999999


### Write to a table in SQLite

In [11]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

db = sqlite3.connect('Wedge_Task3_2021.db')
c = db.cursor()

# input the results of the first query into the WedgeTask3 database as the table Owner_Trans_Yr_Month
input_file2 = "Owner_year_month_trans.txt"
#db = sqlite3.connect("WedgeTask3.db") # connect to the new database
#c = db.cursor()
c.execute('''DROP TABLE IF EXISTS Owner_Trans_Yr_Month''') # remove the table if it already exists
# create the table in the connected database and set up the schema
c.execute('''CREATE TABLE Owner_Trans_Yr_Month (
    Owner_ID INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')
# input the Sales by Date by Hour text data into the established table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO Owner_Trans_Yr_Month (Owner_ID, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion
db.commit()

end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process

3.91191409999999


## Query 3:

3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [12]:

task3_nbr3 = '''SELECT tr.department, dl.dept_name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    upc,
    description,
    tr.department AS dept_no,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-fall2021-92691.Wedge_FULL.transArchive*` AS tr
    
    LEFT OUTER JOIN `wedge-project-fall2021-92691.Wedge_FULL.department_lookup` AS dl ON tr.department = dl.department
    
    WHERE card_no != 3
    AND tr.department != 0
    AND tr.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Year, Month, upc, description, dept_no, dl.dept_name
    ORDER BY description, Year, Month DESC'''    
    
desc_summarized_trans = client.query(task3_nbr3,location="US")

### set up headers

In [13]:
headers = ["department","dept_name","Year","Month","upc","description","dept_no","Sales","Transactions","Items"]

### Create text file of query results

In [14]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

with open("monthly_trans_by_description.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in desc_summarized_trans:
         outfile.write(",".join([str(item) for item in row])+ '\n')
            
end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process            

108.7543053


### Write to a table in SQLite

In [15]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

db = sqlite3.connect('Wedge_Task3_2021.db')
c = db.cursor()

# input the results of the first query into the WedgeTask3 database as the table Item_Description_by_Month
input_file3 = "monthly_trans_by_description.txt"
#db = sqlite3.connect("WedgeTask3.db") # connect to the new database
#c = db.cursor()
c.execute('''DROP TABLE IF EXISTS Item_Description_by_Month''') # remove the table if it already exists
# create the table in the connected database and set up the schema
c.execute('''CREATE TABLE Item_Description_by_Month (
    department FLOAT,
    dept_name TEXT,
    Year TIMESTAMP,
    Month TIMESTAMP,
    upc STRING,
    description STRING,
    dept_no INTEGER,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')
# input the Sales by Date by Hour text data into the established table in the database
with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO Item_Description_by_Month (department, dept_name, Year, Month, upc, description, dept_no, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion
db.commit()

end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process

6.892051199999997
